# Ensemble Model Validation

Comprehensive validation tests for ensemble models.

**Tests:**
1. Simple averaging ensemble
2. Weighted averaging ensemble
3. Stacking ensemble
4. Ensemble vs single model performance
5. Weight optimization
6. Serialization (save/load)
7. Edge cases and error handling
8. Dual goal predictor ensemble

Run this BEFORE using the model in production to catch bugs.

In [ ]:
# Setup
import sys
sys.path.insert(0, '..')

import pandas as pd
import numpy as np
import tempfile
import os
import pickle
from pathlib import Path

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, StackingRegressor
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler

# Test tracking
test_results = []

def record_test(name, passed, message=""):
    status = "✅ PASS" if passed else "❌ FAIL"
    test_results.append({'test': name, 'passed': passed, 'message': message})
    print(f"{status}: {name}")
    if message:
        print(f"       {message}")

print("Validation setup complete!")

## Generate Test Data

In [ ]:
# Create synthetic hockey-like test data
np.random.seed(42)
n = 500

X_data = pd.DataFrame({
    'home_win_pct': np.random.uniform(0.3, 0.7, n),
    'away_win_pct': np.random.uniform(0.3, 0.7, n),
    'home_goals_avg': np.random.uniform(2.5, 3.8, n),
    'away_goals_avg': np.random.uniform(2.5, 3.8, n),
    'home_goals_against_avg': np.random.uniform(2.2, 3.5, n),
    'away_goals_against_avg': np.random.uniform(2.2, 3.5, n),
    'home_pp_pct': np.random.uniform(0.15, 0.28, n),
    'away_pp_pct': np.random.uniform(0.15, 0.28, n),
    'home_rest_days': np.random.randint(1, 5, n),
    'away_rest_days': np.random.randint(1, 5, n),
})

# Create realistic target with known relationships
y_home = (
    X_data['home_goals_avg'] * 0.4 +
    (4 - X_data['away_goals_against_avg']) * 0.3 +
    X_data['home_pp_pct'] * 5 +
    0.3 +  # home advantage
    np.random.normal(0, 0.5, n)
).clip(0, 8).round().astype(int)

y_away = (
    X_data['away_goals_avg'] * 0.4 +
    (4 - X_data['home_goals_against_avg']) * 0.3 +
    X_data['away_pp_pct'] * 5 +
    np.random.normal(0, 0.5, n)
).clip(0, 8).round().astype(int)

# Train/val/test split
train_idx = int(n * 0.6)
val_idx = int(n * 0.8)

X_train = X_data.iloc[:train_idx]
X_val = X_data.iloc[train_idx:val_idx]
X_test = X_data.iloc[val_idx:]

y_home_train = y_home[:train_idx]
y_home_val = y_home[train_idx:val_idx]
y_home_test = y_home[val_idx:]

y_away_train = y_away[:train_idx]
y_away_val = y_away[train_idx:val_idx]
y_away_test = y_away[val_idx:]

# Scale for linear models
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

print(f"Train: {len(X_train)}, Val: {len(X_val)}, Test: {len(X_test)}")
print(f"Home goals range: {y_home.min()} - {y_home.max()}")

## Train Base Models

In [ ]:
# Train base models for ensemble
base_models = {}

# Linear model (needs scaled data)
ridge = Ridge(alpha=1.0)
ridge.fit(X_train_scaled, y_home_train)
base_models['ridge'] = ('scaled', ridge)

# Random Forest
rf = RandomForestRegressor(n_estimators=100, max_depth=8, random_state=42)
rf.fit(X_train, y_home_train)
base_models['rf'] = ('raw', rf)

# Gradient Boosting
gb = GradientBoostingRegressor(n_estimators=100, max_depth=5, random_state=42)
gb.fit(X_train, y_home_train)
base_models['gb'] = ('raw', gb)

print(f"Trained {len(base_models)} base models")

# Get validation predictions
val_predictions = {}
for name, (data_type, model) in base_models.items():
    X = X_val_scaled if data_type == 'scaled' else X_val
    val_predictions[name] = model.predict(X)
    rmse = np.sqrt(mean_squared_error(y_home_val, val_predictions[name]))
    print(f"  {name}: RMSE = {rmse:.4f}")

## Test 1: Simple Averaging Ensemble

In [ ]:
try:
    # Simple average of all predictions
    avg_pred = np.mean(list(val_predictions.values()), axis=0)
    
    avg_rmse = np.sqrt(mean_squared_error(y_home_val, avg_pred))
    avg_mae = mean_absolute_error(y_home_val, avg_pred)
    
    record_test("1a. Simple averaging", True, f"RMSE: {avg_rmse:.4f}, MAE: {avg_mae:.4f}")
    
    # Average should be better than worst individual model
    worst_rmse = max(np.sqrt(mean_squared_error(y_home_val, p)) for p in val_predictions.values())
    better_than_worst = avg_rmse <= worst_rmse
    record_test("1b. Average beats worst model", better_than_worst,
               f"Avg RMSE: {avg_rmse:.4f}, Worst: {worst_rmse:.4f}")
except Exception as e:
    record_test("1. Simple averaging", False, str(e))

## Test 2: Weighted Averaging Ensemble

In [ ]:
try:
    # Calculate weights based on inverse RMSE
    rmses = {name: np.sqrt(mean_squared_error(y_home_val, pred)) 
             for name, pred in val_predictions.items()}
    
    # Inverse RMSE weights
    inv_rmse = {name: 1/r for name, r in rmses.items()}
    total = sum(inv_rmse.values())
    weights = {name: w/total for name, w in inv_rmse.items()}
    
    record_test("2a. Weight calculation", abs(sum(weights.values()) - 1.0) < 0.01,
               f"Weights: {', '.join(f'{k}={v:.3f}' for k,v in weights.items())}")
    
    # Weighted average
    weighted_pred = np.zeros(len(y_home_val))
    for name, pred in val_predictions.items():
        weighted_pred += pred * weights[name]
    
    weighted_rmse = np.sqrt(mean_squared_error(y_home_val, weighted_pred))
    
    # Weighted should be >= simple average (or very close)
    record_test("2b. Weighted average RMSE", weighted_rmse < worst_rmse,
               f"Weighted RMSE: {weighted_rmse:.4f}")
    
    # Better model should have higher weight
    best_model = min(rmses, key=rmses.get)
    best_has_highest_weight = weights[best_model] == max(weights.values())
    record_test("2c. Best model has highest weight", best_has_highest_weight,
               f"Best: {best_model} (weight={weights[best_model]:.3f})")
except Exception as e:
    record_test("2. Weighted averaging", False, str(e))

## Test 3: Stacking Ensemble

In [ ]:
try:
    # Define estimators for stacking
    estimators = [
        ('ridge', Ridge(alpha=1.0)),
        ('rf', RandomForestRegressor(n_estimators=50, max_depth=6, random_state=42)),
        ('gb', GradientBoostingRegressor(n_estimators=50, max_depth=4, random_state=42)),
    ]
    
    # Create stacking regressor
    stacking = StackingRegressor(
        estimators=estimators,
        final_estimator=Ridge(alpha=1.0),
        cv=3,
        n_jobs=-1
    )
    
    # Combine train and val for stacking (it does internal CV)
    X_trainval = pd.concat([X_train, X_val])
    y_trainval = np.concatenate([y_home_train, y_home_val])
    
    stacking.fit(X_trainval, y_trainval)
    record_test("3a. Stacking model fit", True)
    
    # Test set predictions
    stack_pred = stacking.predict(X_test)
    stack_rmse = np.sqrt(mean_squared_error(y_home_test, stack_pred))
    
    record_test("3b. Stacking predictions", len(stack_pred) == len(y_home_test),
               f"Test RMSE: {stack_rmse:.4f}")
    
    # Check final estimator was trained
    record_test("3c. Final estimator trained", hasattr(stacking, 'final_estimator_'))
except Exception as e:
    record_test("3. Stacking ensemble", False, str(e))

## Test 4: Ensemble vs Single Model

In [ ]:
try:
    # Get test predictions from base models
    test_predictions = {}
    for name, (data_type, model) in base_models.items():
        X = X_test_scaled if data_type == 'scaled' else X_test
        test_predictions[name] = model.predict(X)
    
    # Calculate individual RMSEs
    individual_rmses = {name: np.sqrt(mean_squared_error(y_home_test, pred))
                        for name, pred in test_predictions.items()}
    
    # Simple average on test
    test_avg = np.mean(list(test_predictions.values()), axis=0)
    avg_test_rmse = np.sqrt(mean_squared_error(y_home_test, test_avg))
    
    # Weighted average on test (using val weights)
    test_weighted = np.zeros(len(y_home_test))
    for name, pred in test_predictions.items():
        test_weighted += pred * weights[name]
    weighted_test_rmse = np.sqrt(mean_squared_error(y_home_test, test_weighted))
    
    print("Test Set Performance:")
    for name, rmse in individual_rmses.items():
        print(f"  {name}: {rmse:.4f}")
    print(f"  Simple Avg: {avg_test_rmse:.4f}")
    print(f"  Weighted Avg: {weighted_test_rmse:.4f}")
    print(f"  Stacking: {stack_rmse:.4f}")
    
    # Ensemble should be competitive with best individual
    best_individual = min(individual_rmses.values())
    ensemble_competitive = weighted_test_rmse <= best_individual * 1.1
    record_test("4a. Ensemble competitive with best", ensemble_competitive,
               f"Weighted: {weighted_test_rmse:.4f}, Best individual: {best_individual:.4f}")
    
    # Ensemble reduces variance (should be between min and max)
    worst_individual = max(individual_rmses.values())
    reduces_variance = weighted_test_rmse < worst_individual
    record_test("4b. Ensemble beats worst model", reduces_variance,
               f"Weighted: {weighted_test_rmse:.4f}, Worst: {worst_individual:.4f}")
except Exception as e:
    record_test("4. Ensemble vs single model", False, str(e))

## Test 5: Weight Optimization

In [ ]:
try:
    from scipy.optimize import minimize
    
    # Optimize weights on validation set
    pred_matrix = np.column_stack(list(val_predictions.values()))
    
    def ensemble_loss(w):
        w = np.abs(w)  # Ensure positive
        w = w / w.sum()  # Normalize
        pred = pred_matrix @ w
        return mean_squared_error(y_home_val, pred)
    
    # Start with equal weights
    w0 = np.ones(len(base_models)) / len(base_models)
    result = minimize(ensemble_loss, w0, method='Nelder-Mead')
    
    opt_weights = np.abs(result.x)
    opt_weights = opt_weights / opt_weights.sum()
    
    record_test("5a. Weight optimization converged", result.success or result.fun < 1.0,
               f"Optimal weights: {opt_weights.round(3)}")
    
    # Optimized ensemble on validation
    opt_pred = pred_matrix @ opt_weights
    opt_rmse = np.sqrt(mean_squared_error(y_home_val, opt_pred))
    
    # Should be at least as good as heuristic weights
    heuristic_rmse = np.sqrt(mean_squared_error(y_home_val, weighted_pred))
    record_test("5b. Optimized >= heuristic", opt_rmse <= heuristic_rmse * 1.01,
               f"Optimized: {opt_rmse:.4f}, Heuristic: {heuristic_rmse:.4f}")
except ImportError:
    record_test("5. Weight optimization", True, "scipy not available, skipping")
except Exception as e:
    record_test("5. Weight optimization", False, str(e))

## Test 6: Serialization

In [ ]:
try:
    # Save stacking ensemble
    with tempfile.NamedTemporaryFile(suffix='.pkl', delete=False) as f:
        pickle.dump(stacking, f)
        temp_path = f.name
    
    # Load
    with open(temp_path, 'rb') as f:
        loaded_stacking = pickle.load(f)
    
    # Compare predictions
    original_pred = stacking.predict(X_test)
    loaded_pred = loaded_stacking.predict(X_test)
    
    match = np.allclose(original_pred, loaded_pred)
    record_test("6a. Stacking save/load", match,
               f"Max diff: {np.abs(original_pred - loaded_pred).max():.6f}")
    
    os.unlink(temp_path)
    
    # Save weighted ensemble components
    ensemble_data = {
        'models': base_models,
        'weights': weights,
        'scaler': scaler,
    }
    
    with tempfile.NamedTemporaryFile(suffix='.pkl', delete=False) as f:
        pickle.dump(ensemble_data, f)
        temp_path = f.name
    
    with open(temp_path, 'rb') as f:
        loaded_data = pickle.load(f)
    
    weights_match = loaded_data['weights'] == weights
    record_test("6b. Weights preserved", weights_match)
    
    os.unlink(temp_path)
except Exception as e:
    record_test("6. Serialization", False, str(e))

## Test 7: Edge Cases

In [ ]:
# Test 7a: Single sample prediction
try:
    single_pred = stacking.predict(X_test.iloc[[0]])
    record_test("7a. Single sample prediction", len(single_pred) == 1,
               f"Prediction: {single_pred[0]:.4f}")
except Exception as e:
    record_test("7a. Single sample prediction", False, str(e))

# Test 7b: Two model ensemble
try:
    small_estimators = [
        ('rf', RandomForestRegressor(n_estimators=10, random_state=42)),
        ('gb', GradientBoostingRegressor(n_estimators=10, random_state=42)),
    ]
    
    small_stacking = StackingRegressor(
        estimators=small_estimators,
        final_estimator=LinearRegression(),
        cv=2
    )
    small_stacking.fit(X_train, y_home_train)
    small_pred = small_stacking.predict(X_test)
    
    record_test("7b. Two model ensemble", len(small_pred) == len(X_test))
except Exception as e:
    record_test("7b. Two model ensemble", False, str(e))

# Test 7c: All weights equal
try:
    equal_weights = {name: 1/len(val_predictions) for name in val_predictions}
    equal_pred = np.mean(list(val_predictions.values()), axis=0)
    
    # Compare with manually weighted
    manual_pred = np.zeros(len(y_home_val))
    for name, pred in val_predictions.items():
        manual_pred += pred * equal_weights[name]
    
    match = np.allclose(equal_pred, manual_pred)
    record_test("7c. Equal weights = simple average", match)
except Exception as e:
    record_test("7c. Equal weights = simple average", False, str(e))

# Test 7d: Predictions reasonable
try:
    test_pred = stacking.predict(X_test)
    min_pred, max_pred = test_pred.min(), test_pred.max()
    
    reasonable = min_pred >= -1 and max_pred <= 10
    record_test("7d. Reasonable prediction range", reasonable,
               f"Range: [{min_pred:.2f}, {max_pred:.2f}]")
except Exception as e:
    record_test("7d. Reasonable prediction range", False, str(e))

## Test 8: Dual Goal Predictor Ensemble

In [ ]:
try:
    # Train separate ensembles for home and away
    estimators = [
        ('ridge', Ridge(alpha=1.0)),
        ('rf', RandomForestRegressor(n_estimators=50, max_depth=6, random_state=42)),
    ]
    
    X_trainval = pd.concat([X_train, X_val])
    y_home_trainval = np.concatenate([y_home_train, y_home_val])
    y_away_trainval = np.concatenate([y_away_train, y_away_val])
    
    home_ensemble = StackingRegressor(
        estimators=estimators,
        final_estimator=Ridge(),
        cv=3
    )
    home_ensemble.fit(X_trainval, y_home_trainval)
    
    away_ensemble = StackingRegressor(
        estimators=estimators,
        final_estimator=Ridge(),
        cv=3
    )
    away_ensemble.fit(X_trainval, y_away_trainval)
    
    home_pred = home_ensemble.predict(X_test)
    away_pred = away_ensemble.predict(X_test)
    
    home_rmse = np.sqrt(mean_squared_error(y_home_test, home_pred))
    away_rmse = np.sqrt(mean_squared_error(y_away_test, away_pred))
    
    # Combined
    all_pred = np.concatenate([home_pred, away_pred])
    all_actual = np.concatenate([y_home_test, y_away_test])
    combined_rmse = np.sqrt(mean_squared_error(all_actual, all_pred))
    
    record_test("8a. Dual ensemble training", True,
               f"Home RMSE: {home_rmse:.4f}, Away RMSE: {away_rmse:.4f}")
    record_test("8b. Combined performance", combined_rmse < 2.0,
               f"Combined RMSE: {combined_rmse:.4f}")
except Exception as e:
    record_test("8. Dual goal predictor ensemble", False, str(e))

## Test 9: Cross-Validation Stability

In [ ]:
try:
    estimators = [
        ('ridge', Ridge(alpha=1.0)),
        ('rf', RandomForestRegressor(n_estimators=50, max_depth=6, random_state=42)),
    ]
    
    stacking_cv = StackingRegressor(
        estimators=estimators,
        final_estimator=Ridge(),
        cv=3
    )
    
    kfold = KFold(n_splits=5, shuffle=True, random_state=42)
    cv_scores = cross_val_score(stacking_cv, X_data, y_home, cv=kfold,
                                scoring='neg_root_mean_squared_error')
    
    mean_rmse = -cv_scores.mean()
    std_rmse = cv_scores.std()
    cv_ratio = std_rmse / mean_rmse
    
    is_stable = cv_ratio < 0.3
    record_test("9. Cross-validation stability", is_stable,
               f"RMSE: {mean_rmse:.4f} (+/- {std_rmse:.4f}), CV ratio: {cv_ratio:.2%}")
except Exception as e:
    record_test("9. Cross-validation stability", False, str(e))

## Test Summary

In [ ]:
# Summary
print("\n" + "=" * 60)
print(" ENSEMBLE VALIDATION SUMMARY")
print("=" * 60)

results_df = pd.DataFrame(test_results)
passed = results_df['passed'].sum()
total = len(results_df)

print(f"\nPassed: {passed}/{total} ({passed/total*100:.1f}%)")

if passed < total:
    print("\n❌ FAILED TESTS:")
    for _, row in results_df[~results_df['passed']].iterrows():
        print(f"   - {row['test']}: {row['message']}")
else:
    print("\n✅ All tests passed!")

# Show all results
print("\nDetailed Results:")
print(results_df.to_string(index=False))